# Particle Categorical

In [1]:
import tensorflow as tf

import tensorflow.keras as keras
import tensorflow.keras.backend as K
# from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import Flatten, Reshape, Lambda
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras import Model

import os
import os.path as osp
import sys

import numpy as np
#from scipy import linalg as LA

import matplotlib
import matplotlib.pyplot as plt

# %load_ext autoreload
# %autoreload 2

from utils.tf_sinkhorn import ground_distance_tf_nograd, sinkhorn_knopp_tf_scaling_stabilized_class
import utils.VAE_model_tools
from utils.VAE_model_tools import build_and_compile_annealing_vae, betaVAEModel, reset_metrics

import pandas
import matplotlib.pyplot as plt

import h5py
import pickle


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [3]:
def create_dir(dir_path):
    ''' Creates a directory (or nested directories) if they don't exist.
    '''
    if not osp.exists(dir_path):
        os.makedirs(dir_path)

    return dir_path

output_dir = './data/'

## Generate training data

Input data is 2D, generated in the shape of a banana (plotted below) defined by two gaussians with widths 1 and 0.1. The VAE will be tasked with reconstructing the 2D location of the input points, using euclidean distance as the reconstruction error. The latent space is 2D, so can in principle easily encode everything about the input. This will be regulated by the variational latent space

In [4]:
# path to file
fn_sig =  'E:/signal_processed.h5'
fn_bg =  'E:/background_processed.h5'

In [5]:
# Option 1: Load everything into memory
df_sig = pandas.read_hdf(fn_sig,stop=100000)
df_bg = pandas.read_hdf(fn_bg,stop=1000000)
print(df_bg.shape)
print(df_sig.shape)
print("Memory in GB:",sum(df_sig.memory_usage(deep=True)) / (1024**3)+sum(df_bg.memory_usage(deep=True)) / (1024**3))

(504219, 300)
(74461, 300)
Memory in GB: 1.2977620959281921


In [6]:
sig_data = df_sig.values.reshape((-1,100,3))
bg_data = df_bg.values.reshape((-1,100,3))

HT_sig = np.sum(sig_data[:,:,0],axis=-1)
sig_data[:,:,0] = sig_data[:,:,0]/HT_sig[:,None]

HT_bg = np.sum(bg_data[:,:,0],axis=-1)
bg_data[:,:,0] = bg_data[:,:,0]/HT_bg[:,None]

sig_input = np.zeros((len(sig_data),100,4))
sig_input[:,:,:2] = sig_data[:,:,:2]
sig_input[:,:,2] = np.cos(sig_data[:,:,-1])
sig_input[:,:,3] = np.sin(sig_data[:,:,-1])

bg_input = np.zeros((len(bg_data),100,4))
bg_input[:,:,:2] = bg_data[:,:,:2]
bg_input[:,:,2] = np.cos(bg_data[:,:,-1])
bg_input[:,:,3] = np.sin(bg_data[:,:,-1])

data_x = np.append(bg_input,sig_input,axis=0)
data_y = np.append(bg_data,sig_data,axis=0)
perms = np.random.permutation(len(data_x))
data_x = data_x[perms]
data_y = data_y[perms]

train_x = data_x[:500000]
train_y = data_y[:500000]
valid_x = data_x[500000:]
valid_y = data_y[500000:]

In [7]:
experiment_name = 'particle_32_cats_2'
train_output_dir = create_dir(osp.join(output_dir, experiment_name))
vae, encoder, decoder = build_and_compile_annealing_vae(optimizer=keras.optimizers.Adam(lr=0.001,clipnorm=0.1),
                                    encoder_conv_layers = [512,512,512,512],
                                    dense_size = 512,
                                    decoder = [512,512,512,512],
                                    numItermaxinner = 10,
                                    numIter=10,
                                    reg_init = 1.,
                                    reg_final = 0.01,
                                    stopThr=1e-3)

batch_size=100
save_period=2

reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=0)
modelcheckpoint = keras.callbacks.ModelCheckpoint(train_output_dir + '/model_weights_{epoch:02d}.hdf5', save_freq = save_period*5000, save_weights_only=True)
reset_metrics_inst = reset_metrics()

callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,
            modelcheckpoint,
            reset_metrics_inst]

Model: "CNN-VAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 100, 4)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 100, 512)     2560        inputs[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 100, 512)     0           conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 100, 512)     262656      re_lu[0][0]                      
____________________________________________________________________________________________

In [8]:
batch_size=100
save_period=2

reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=0)
modelcheckpoint = keras.callbacks.ModelCheckpoint(train_output_dir + '/model_weights_{epoch:02d}.hdf5', save_freq = save_period*5000, save_weights_only=True)
reset_metrics_inst = reset_metrics()

callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,
            modelcheckpoint,
            reset_metrics_inst]

vae.beta.assign(0.01)
vae.alpha.assign(1.)
numbatches = 5000

K.set_value(vae.optimizer.lr,1e-4)
epochs = 1000


history = vae.fit(x=train_x[:numbatches*batch_size], y=train_y[:numbatches*batch_size], batch_size=batch_size,
                epochs=epochs,verbose=1,#initial_epoch=int(vae.optimizer.iterations/numbatches),
                validation_data = (valid_x[:10*batch_size],valid_y[:10*batch_size]),
                callbacks = callbacks
              )

# tf.saved_model.save(vae, train_output_dir + '/mymodel.hdf5')

Epoch 1/1000
  42/5000 [..............................] - ETA: 52:43 - loss: 10829.5432 - recon_loss: 1.0539 - KL loss: 164.7922 - KL bern loss: 125.9271 - beta: 0.0100 - alpha: 1.0000

KeyboardInterrupt: 